In [1]:
import random

In [3]:
class Grafo:
    
    def __init__(self):
        self.grafo = {}
    
    def import_m(self, M='Matriz.txt'):
        """
        Importa una matris de un documento de texto
        """
        with open(M, "r" , encoding="utf-8") as m:
            Matriz = m.read().split('\n')
        
        return Matriz
        
    def matrix_2_dict(self,data): 
        for linea in data:
            origen, destino, peso = linea.split()
            peso = int(peso)

            if origen not in self.grafo:
                self.grafo[origen] = {}
            if destino not in self.grafo:
                self.grafo[destino] = {}
    
            self.grafo[origen][destino] = peso
            self.grafo[destino][origen] = peso

        return self.grafo
    
    def vrp(self, grafo,nodo_c):
        
        G = nx.Graph(grafo)
        particiones = list(kernighan_lin_bisection(G))
        print(particiones)
        l =[]
        for particion in particiones:
            #print(type(particion))
            self.grafo = deepcopy(grafo)
            #print(self.grafo)
            if nodo_c in particion:
                particion.remove(nodo_c)

            for nodo in particion:
                if nodo in self.grafo:
                    del self.grafo[nodo]
                for nodo_conectado in self.grafo:
                    if nodo in self.grafo[nodo_conectado]:
                        del self.grafo[nodo_conectado][nodo]
            #print(self.grafo)

            l.append(self.grafo)
    
        return l
    
    def floyd_warshall(self,grafo):
        # Crear una copia del grafo para no modificar el original
        distancias = {nodo: {otro_nodo: float('inf') for otro_nodo in grafo} for nodo in grafo}
        
        # Inicializar las distancias conocidas (distancia entre un nodo y sí mismo es 0)
        for nodo in grafo:
            distancias[nodo][nodo] = 0
        
        # Actualizar las distancias con los valores del grafo original
        for nodo in grafo:
            for vecino in grafo[nodo]:
                distancias[nodo][vecino] = grafo[nodo][vecino]
        
        # Aplicar el algoritmo de Floyd-Warshall
        for intermedio in grafo:
            for origen in grafo:
                for destino in grafo:
                    # Si el camino a través del nodo intermedio es más corto, actualizar
                    if distancias[origen][destino] > distancias[origen][intermedio] + distancias[intermedio][destino]:
                        distancias[origen][destino] = distancias[origen][intermedio] + distancias[intermedio][destino]
        
        return distancias
 


In [4]:
if __name__ == "__main__":
    G=Grafo()
    grafo= G.matrix_2_dict(G.import_m())
    print(grafo)
    grafo= G.floyd_warshall(G.matrix_2_dict(G.import_m()))
    print(grafo)

{'A': {'B': 2, 'C': 7, 'D': 3}, 'B': {'A': 2, 'C': 8, 'D': 4}, 'C': {'A': 7, 'B': 8, 'D': 1}, 'D': {'A': 3, 'B': 4, 'C': 1}}
{'A': {'A': 0, 'B': 2, 'C': 4, 'D': 3}, 'B': {'A': 2, 'B': 0, 'C': 5, 'D': 4}, 'C': {'A': 4, 'B': 5, 'C': 0, 'D': 1}, 'D': {'A': 3, 'B': 4, 'C': 1, 'D': 0}}


In [5]:
def inicializar_poblacion(grafo, nodo_inicial, numero_p):
    """
    Esta funcion crea una poblacion inicial.
    Entrada: grafo, nodo que inicia y numero de poblacion 
    Salidad : rutas de la poblacion. Ejemplo: [A,B,C,D,A]
    """
    poblacion = []
    nodos = list(grafo.keys())

    for _ in range(numero_p):
        ruta = [nodo_inicial]
        nodo_actual = nodo_inicial

        while len(ruta) < len(nodos):
            # Seleccionar un nodo vecino aleatorio
            vecinos = list(grafo[nodo_actual])
            nodo_siguiente = random.choice(vecinos)

            # Evitar ciclos en la ruta
            if nodo_siguiente not in ruta:
                ruta.append(nodo_siguiente)
                nodo_actual = nodo_siguiente

        # Completar el ciclo volviendo al nodo inicial
        ruta.append(nodo_inicial)
        poblacion.append(ruta)

    return poblacion


In [6]:
poblacion = inicializar_poblacion(grafo,"A",4)
print(poblacion)

[['A', 'C', 'B', 'D', 'A'], ['A', 'D', 'B', 'C', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'B', 'C', 'D', 'A']]


In [7]:
def calcular_peso_ruta(grafo, ruta):
    peso_total = sum(grafo[ruta[i]][ruta[i + 1]] for i in range(len(ruta) - 1))
    return peso_total


def evaluacion(grafo, poblacion):
    
    evaluaciones = []

    for ruta in poblacion:
        peso_ruta = calcular_peso_ruta(grafo, ruta)
        evaluaciones.append((ruta, peso_ruta))

    return evaluaciones


In [8]:
evaluacion1 = evaluacion(grafo,poblacion)
print(evaluacion1)

[(['A', 'C', 'B', 'D', 'A'], 16), (['A', 'D', 'B', 'C', 'A'], 16), (['A', 'D', 'C', 'B', 'A'], 11), (['A', 'B', 'C', 'D', 'A'], 11)]


In [9]:
def seleccion(evaluaciones, porcentaje_de_seleccion_de_poblacion=0.5):
    
    evaluaciones_ordenadas = sorted(evaluaciones, key=lambda x: x[1])  # Ordenar evaluaciones por peso

    # Calcular la cantidad de individuos a seleccionar
    cantidad_a_seleccionar = int(len(evaluaciones) * porcentaje_de_seleccion_de_poblacion)

    # Seleccionar los mejores individuos según el porcentaje
    seleccionados = evaluaciones_ordenadas[:cantidad_a_seleccionar]

    return seleccionados

In [10]:
porcentaje_de_seleccion_de_poblacion =0.5

In [11]:
seleccion1 = seleccion(evaluacion1,porcentaje_de_seleccion_de_poblacion)
print(seleccion1)

[(['A', 'D', 'C', 'B', 'A'], 11), (['A', 'B', 'C', 'D', 'A'], 11)]


In [12]:
def one_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, min(len(parent1), len(parent2)-1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2


def partially_mapped_crossover(parent1, parent2):
    #global mapeo1,mapeo2,child1,child2,hay_duplicados1,hay_duplicados2,indices_repetidos
    # Selecciona un punto de cruce aleatorio
    crossover_point = random.randint(1, min(len(parent1), len(parent2))-1)
    # Realiza el cruce para generar a los hijos
    child1 = parent1[1:crossover_point] + parent2[crossover_point:-1]
    child2 = parent2[1:crossover_point] + parent1[crossover_point:-1]
    # Crea mapeos entre los elementos de los padres
    mapeo1 = {p1: p2 for p1, p2 in zip(parent1[1:-1], parent2[1:-1])}
    mapeo2 = {p2: p1 for p2, p1 in zip(parent2[1:-1], parent1[1:-1])}

    # Verifica si hay elementos duplicados en los hijos
    hay_duplicados1 =  len(child1) != len(set(child1))
    hay_duplicados2 = len(child2) != len(set(child2))
    
    # Corrige duplicados en el primer hijo
    while hay_duplicados1:
        indices_repetidos = [item for item in child1 if child1.count(item) > 1]
        for item in indices_repetidos:
            index = child1.index(item)
            if item in mapeo1:
                child1[index] = mapeo1[item]
                indices_repetidos.remove(item)
        hay_duplicados1 = len(child1) != len(set(child1)) 
    # Corrige duplicados en el segundo hijo   
    while hay_duplicados2:
        indices_repetidos = [item for item in child2 if child2.count(item) > 1]
        for item in indices_repetidos:
            index = child2.index(item)
            if item in mapeo2:
                child2[index] = mapeo2[item]
                indices_repetidos.remove(item)
        hay_duplicados2 = len(child2) != len(set(child2)) 
    
    child1.append(parent1[0]);child1.insert(0,parent1[0])
    child2.append(parent2[0]);child2.insert(0,parent1[0])
    return child1, child2

def recombinacion(seleccionados):
    hijos = []
    cantidad_seleccionados = len(seleccionados)+(len(seleccionados)*porcentaje_de_seleccion_de_poblacion)

    while len(hijos) < cantidad_seleccionados:
        padre1, padre2 = random.sample(seleccionados, 2)
        hijo1, hijo2 = partially_mapped_crossover(padre1[0], padre2[0])

        hijos.append(hijo1)
        hijos.append(hijo2)

    return hijos


In [13]:
recombinacion1 = recombinacion(seleccion1)
print(recombinacion1)

[['A', 'B', 'C', 'D', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'B', 'C', 'D', 'A']]


In [13]:
def mutacion(recombinacion, probabilidad_mutacion=0.5):
    hijos_mutados = []
    
    for hijo in recombinacion:
        if random.random() < probabilidad_mutacion:
            hijo_mutado = hijo[:]
                      
            nodos_mutables = list(hijo_mutado[1:-1])
            
            for i in range(1, len(hijo_mutado) - 1):
                
                nuevo_nodo = random.choice(nodos_mutables)
                nodos_mutables.remove(nuevo_nodo)
                hijo_mutado[i] = nuevo_nodo
            
            hijos_mutados.append(hijo_mutado)
        else:
            hijos_mutados.append(hijo)
    
    return hijos_mutados


In [14]:
mutacion1 = mutacion(recombinacion1)
print(mutacion1)

[['A', 'C', 'B', 'D', 'A'], ['A', 'C', 'D', 'B', 'A'], ['A', 'B', 'C', 'D', 'A'], ['A', 'B', 'D', 'C', 'A']]


In [15]:
def condicion_termino(grafo, evaluaciones, numero_i=4):
    evaluacion1 = evaluaciones[:]
    seleccion1 = seleccion(evaluacion1, porcentaje_de_seleccion_de_poblacion=0.5)
    
    while numero_i != 0:
        print(f'Selección de padres: {seleccion1}\n')
        
        recombinacion1 = recombinacion(seleccion1)
        print(f'Cruce de padres (Hijos): {recombinacion1}\n')
        
        mutacion1 = mutacion(recombinacion1)
        print(f'Mutación de hijos: {mutacion1}\n')
        
        evaluacion_hijos = evaluacion(grafo, mutacion1)
        print(f'Evaluación de los hijos: {evaluacion_hijos}\n')
        
        seleccion_hijos = seleccion(evaluacion_hijos, porcentaje_de_seleccion_de_poblacion=0.5)
        seleccion_padres_hijos = seleccion_hijos + seleccion1
        seleccion1 = seleccion(seleccion_padres_hijos, porcentaje_de_seleccion_de_poblacion=0.5)
        
        numero_i -= 1
    print(f"seleccion_padres_hijos{seleccion_padres_hijos}")
    print(f'Solución encontrada: {seleccion1}')

In [16]:
condicion_termino(grafo, evaluacion1)

Selección de padres: [(['A', 'B', 'D', 'C', 'A'], 11), (['A', 'D', 'C', 'B', 'A'], 11)]

Cruce de padres (Hijos): [['A', 'D', 'C', 'B', 'A'], ['A', 'B', 'D', 'C', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'B', 'D', 'C', 'A']]

Mutación de hijos: [['A', 'D', 'C', 'B', 'A'], ['A', 'C', 'D', 'B', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'B', 'D', 'C', 'A']]

Evaluación de los hijos: [(['A', 'D', 'C', 'B', 'A'], 11), (['A', 'C', 'D', 'B', 'A'], 11), (['A', 'D', 'C', 'B', 'A'], 11), (['A', 'B', 'D', 'C', 'A'], 11)]

Selección de padres: [(['A', 'D', 'C', 'B', 'A'], 11), (['A', 'C', 'D', 'B', 'A'], 11)]

Cruce de padres (Hijos): [['A', 'C', 'D', 'B', 'A'], ['A', 'D', 'C', 'B', 'A'], ['A', 'C', 'D', 'B', 'A'], ['A', 'D', 'C', 'B', 'A']]

Mutación de hijos: [['A', 'C', 'D', 'B', 'A'], ['A', 'C', 'D', 'B', 'A'], ['A', 'C', 'B', 'D', 'A'], ['A', 'D', 'C', 'B', 'A']]

Evaluación de los hijos: [(['A', 'C', 'D', 'B', 'A'], 11), (['A', 'C', 'D', 'B', 'A'], 11), (['A', 'C', 'B', 'D', 'A'], 16), (['A', 'D'

In [1]:
child1 = [['a', 'b'], ['c', 'd'], ['e', 'b']]
elemento_a = 'a'
nuevo_valor = 'z'

for sublist in child1:
    if elemento_a in sublist:
        index = sublist.index(elemento_a)
        sublist[index] = nuevo_valor

print(child1)


[['z', 'b'], ['c', 'd'], ['e', 'b']]


In [2]:
child1 = [('a', 'b'), ('c', 'd'), ('e', 'b')]
elemento_a = 'a'
nuevo_valor = 'z'

for tupla in child1:
    if elemento_a in tupla:
        index = tupla.index(elemento_a)
        lista_aux = list(tupla)
        lista_aux[index] = nuevo_valor
        child1[child1.index(tupla)] = tuple(lista_aux)

print(child1)


[('z', 'b'), ('c', 'd'), ('e', 'b')]
